# Notebook 4: Adding Image Post-Processing

Example: Acquire Process Decide
Acquisitions, Processes, and decisions cann be prebuilt from libraries and stiched 

The AcquisitonPLugin is a set of variables and callback functions that can be used to acquire a dataset.

The PostProcesingPipeline takes a dataset and returns ProcessedData. 

The Deicision takes ProcessedData and returns an acquisition.

This 3 step process enables advanced automation of image acquistion. 

In [4]:
import sys,os
sys.path.append('source')

In [5]:
from source.apd import *

## Example 4.1: Using the APD System to run an AcquisitionPlugin-PostProcessPipeline-Decision (APD)

In [6]:
from source.apd import *
acqLibrary=AcquisitionPluginLibrary()
acquisition=acqLibrary.xySequence([[0,0]])

process=PostProcessPipeline()
process.add('mean')

decision=DecisionRepeatAcquisition()

apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') 
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=DecisionRepeatAcquisition())
apdSystem.run()


Exception: Couldn't create Core. Is Micro-Manager running and is the ZMQ server on {port} option enabled?

## Example 4.X: Writing your own PostProcess to print Hello World

This section describes how to make your own PostProcessNode using a function callback.

The first section describes the callback schema for process node functions, and the secon section shows how to run the callbacks.


In [11]:
from source.postprocessors import *

# Section 1: making the Hello and World callback functions

def hello(self,chunks,acq,*args,**kwargs):
    output={}
    return (chunks,output)

def world(self,chunks,acq,*args,**kwargs):
    output={}
    print('World ')
    return (chunks, output)

# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=hello))
processors.addNode(PostProcessNode(function=world))

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_1') # Load Sample dataset
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing

apdSystem=



Dataset opened                 
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}]}
Hello 
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}]}
World


In [22]:
def cropImage(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        chunks[i]=chunks[i][10:,:-10] 
    return (chunks,output)

def averageImageValue(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=np.mean(chunks[i])
    return (chunks,output)


# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=cropImage))
processors.addNode(PostProcessNode(function=averageImageValue))

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_1') # Load Sample dataset
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processors.process(data,acq)

Dataset opened                 
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}]}
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}]}
(512, 512)


## Example X.X: Using Cell Detection on Two Color Images

In [ ]:
def detectCells(self,chunks,acq,*args,**kwargs):
    output={}
    detector=CellDetectorCellMask()
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=detector.process(chunks[i])
    return (chunks,output)

# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=detectCells,squish_axes='channel')) # squish_axes combines images long the channel dimension

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_5') # Load Sample dataset thats is 2 channel
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processedData=processors.process(data,acq)

Dataset opened                 
{frozenset({('position', 0), ('z', 0)}): [{'position': 0, 'z': 0, 'channel': 'DAPI'}]}
{frozenset({('position', 0), ('z', 0)}): [{'position': 0, 'z': 0, 'channel': 'DAPI'}, {'channel': 'Cy5', 'position': 0, 'z': 0}]}
(512, 512)
(512, 512)


ADD DISTRIBUTED COMPUTING!!!!!

TRANSFRER runners to BRIANS 